In [5]:
import requests
import plotly.express as pe
import pandas as pd
# headers={
#     accept:
# }

In [2]:


lifestyles_posts = requests.get('https://www.mygreenatlas.com/wp-json/wp/v2/lifestyles')

impact_data = []

for lifestyle in lifestyles_posts.json():
    id = lifestyle['id']

    impact_resp = requests.get(f'https://www.mygreenatlas.com/wp-json/greenatlas/v1/calculator/{id}')

    name = lifestyle['title']['rendered']
    link = lifestyle['link']
    
    impact = impact_resp.json()
    if impact['IsDefined']:
        impact_data.append({
            "name": name,
            "link": link,
            **impact
        })

print(impact_data)


[{'name': 'Sustainable Birthday Parties', 'link': 'https://www.mygreenatlas.com/lifestyles/sustainablebirthdayparties/', 'DollarsPerYearBenefit': {'Min': 0, 'Max': 0}, 'DollarsUpfrontBenefit': {'Min': 0, 'Max': 0}, 'TonsCO2ePerYearBenefit': {'Min': 0.4564979999999999, 'Max': 0.8477819999999999}, 'TonsCO2eUpfrontBenefit': {'Min': 0, 'Max': 0}, 'HoursPerYearBenefit': {'Min': 0, 'Max': -6}, 'HoursUpfrontBenefit': {'Min': 0, 'Max': 0}, 'IsDefined': True}, {'name': 'Sustainable Coffee', 'link': 'https://www.mygreenatlas.com/lifestyles/sustainablecoffee/', 'DollarsPerYearBenefit': {'Min': 0, 'Max': 0}, 'DollarsUpfrontBenefit': {'Min': 10, 'Max': 50}, 'TonsCO2ePerYearBenefit': {'Min': 0.0240646, 'Max': 0.0446914}, 'TonsCO2eUpfrontBenefit': {'Min': 0, 'Max': 0}, 'HoursPerYearBenefit': {'Min': 0, 'Max': 0}, 'HoursUpfrontBenefit': {'Min': 0, 'Max': 0}, 'IsDefined': True}, {'name': 'Subscribe to a Solar Garden', 'link': 'https://www.mygreenatlas.com/lifestyles/subscribetoasolargarden/', 'DollarsP

In [10]:
impact_df = pd.json_normalize(impact_data)

In [11]:

plt1 = pe.scatter(data_frame=impact_df, x='HoursPerYearBenefit.Min', y='TonsCO2ePerYearBenefit.Min', error_x='HoursPerYearBenefit.Max')
plt1.show()